In [ ]:
!curl https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4073  100  4073    0     0  19261      0 --:--:-- --:--:-- --:--:-- 19488


In [ ]:
# import requests

# docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()

import tqdm
import json

with open("../../../01-intro/documents.json") as f:
    documents_raw = json.load(f)


documents = []

for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [91]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://127.0.0.1:9200")
es_client.info()

ObjectApiResponse({'name': 'DESKTOP-BFJIGOM', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'ylVraSS5T3aq0OzhERU41Q', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

index_name = "course-faq"


es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-faq'})

In [ ]:
def es_insert_docs(documents, es_client, es_index):
    try:
        for doc in tqdm.tqdm(documents):
            es_client.index(index=es_index, document=doc)
        return True
    except Exception as e:
        print(e)

100%|██████████| 948/948 [00:03<00:00, 290.34it/s]


In [ ]:
def es_search(
    query, es_client, es_index, es_boost=None, es_filter=None, results_limit=5
):
    es_query = {
        "size": results_limit,
        "query": {
            "bool": {
                **{
                    "must": {
                        "multi_match": {
                            "query": query,
                            "fields": ["question^4", "text"],
                            # "fields": ["question"],
                            "type": "best_fields",
                        }
                    },
                },
                **({"filter": {"term": es_filter}} if es_filter else {}),
            }
        },
    }
    es_results = es_client.search(index=es_index, body=es_query)

    # es_results = es_client.search(
    #     index=es_index,
    #     query=query,
    #     filter_dict=es_filter,
    #     boost_dict=es_boost,
    #     num_results=5,
    # )
    return es_results

In [158]:
query = "How do execute a command on a Kubernetes pod?"


# def rag(query):
#     es_results = es_search(query)
#     prompt = build_prompt(query, es_results)
#     answer = llm(prompt)
#     return answer


es_boost = {"question": 4}
es_filter = {"course": "data-engineering-zoomcamp"}
es_results = es_search(query, es_client, es_index=index_name, es_boost=es_boost)

# es_results.keys(), es_results["hits"].keys()
# es_results["hits"]["hits"][1]["_source"]
es_results["hits"]["hits"][0]['_score'] # q3

44.50556

In [ ]:
query = "How do copy a file to a Docker container?"

es_boost = {"question": 4}
es_filter = {"course": "machine-learning-zoomcamp"}
es_results = es_search(query, es_client, es_index=index_name, es_boost=es_boost, es_filter=es_filter, results_limit=3)
[res['_source']['question'] for res in es_results["hits"]["hits"]][-1] # q4

['How do I debug a docker container?',
 'How do I copy files from my local machine to docker container?',
 'How do I copy files from a different folder into docker container’s working directory?']

In [ ]:
doc_context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join(
    [
        doc_context_template.format(**res["_source"])
        for res in es_results["hits"]["hits"]
    ]
)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)
prompt.__len__() # q5

1446

In [ ]:

import tiktoken


encoding = tiktoken.encoding_for_model("gpt-4o")
prompt_tokens = encoding.encode(prompt)
prompt_tokens.__len__() # q6

encoding.decode_single_token_bytes(63842)

b"You're"